# train_validate.py test
---

In [1]:
%config Completer.use_jedi = False
# import libraries
import torch
import numpy as np

torch.manual_seed(7)

### Train/Validation/Test splits with random_split

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
full_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# split training dataset in training and validation
n = len(full_data)
n_val = int(n * valid_size)
train_data, valid_data = torch.utils.data.random_split(full_data, [n-n_val, n_val])

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)

C:\Users\edgar\anaconda3\envs\dlnd-env\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
assert len(train_loader.sampler) == 8000
assert len(valid_loader.sampler) == 2000

---
## Define the Network [Architecture](http://pytorch.org/docs/stable/nn.html)

The architecture will be responsible for seeing as input a 784-dim Tensor of pixel values for each image, and producing a Tensor of length 10 (our number of classes) that indicates the class scores for an input image. This particular example uses two hidden layers and dropout to avoid overfitting.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 512
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(28 * 28, hidden_1)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_1, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


###  Specify [Loss Function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [Optimizer](http://pytorch.org/docs/stable/optim.html)

It's recommended that you use cross-entropy loss for classification. If you look at the documentation (linked above), you can see that PyTorch's cross entropy function applies a softmax funtion to the output layer *and* then calculates the log loss.

In [5]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

### Train using train_validate.py function

In [6]:
import torchmetrics
from train_validate_test_module import train_validate

%matplotlib inline

train_losses = []
valid_losses = []
def on_epoch_end(model, epoch, epoch_train_loss, epoch_valid_loss, best_valid_loss, train_metrics, valid_metrics):
    train_losses.append(epoch_train_loss)
    valid_losses.append(epoch_valid_loss)    
    print(f'Epoch {epoch}. Val loss: {round(epoch_valid_loss, 4)}. Val Accuracy: {round(valid_metrics["Acc"] * 100, 1)}%')
    if epoch_valid_loss > best_valid_loss:                    
        # Stop if validation loss has increased
        print(f'   Validation loss increased from {best_valid_loss} to {epoch_valid_loss}. Stopping...')
    else:    
        print('   Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(best_valid_loss, epoch_valid_loss))

best_model_state = train_validate(model, train_loader, valid_loader, criterion, optimizer, epochs=2, on_epoch_end=on_epoch_end, metric_factories={'Acc': torchmetrics.Accuracy}, stop_on_loss_increase=True )
model.load_state_dict(best_model_state)
torch.save(model.state_dict(), 'model.pt')

print('Done. Saved to model.pt')


valid: 100%|██████████| 100/100 [00:00<00:00, 174.52it/s]


Epoch 0. Val loss: 1.0574. Val Accuracy: 81.3%
   Validation loss decreased (inf --> 1.057446).  Saving model ...


valid: 100%|██████████| 100/100 [00:00<00:00, 169.49it/s]


Epoch 1. Val loss: 0.5999. Val Accuracy: 86.9%
   Validation loss decreased (1.057446 --> 0.599906).  Saving model ...
Done. Saved to model.pt


In [7]:
train_losses

[1.699950288683176, 0.7986229224503041]

In [8]:
valid_losses

[1.0574456214904786, 0.5999055328965187]

In [9]:
assert np.allclose(train_losses, [1.699950288683176, 0.7986229224503041])

In [10]:
assert np.allclose(valid_losses, [1.0574456214904786, 0.5999055328965187])

In [11]:
print('All tests passed!')

All tests passed!


In [12]:
from train_validate_test_module import test

test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# prepare data loaders
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

avg_loss, computed_metrics = test(model, test_loader, criterion, optimizer, {'Acc': torchmetrics.Accuracy})

test: 100%|██████████| 500/500 [00:03<00:00, 136.31it/s]


In [13]:
assert(np.allclose(avg_loss, 0.6082273083031178))


In [14]:
assert(np.allclose(computed_metrics['Acc'], 0.8621000051498413))